In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
missing_values = ["n/a", "na", "NaN","nan"] #these values will be considered as missing values
biden = pd.read_csv('/kaggle/input/us-election-2020-tweets/hashtag_joebiden.csv',na_values = missing_values)

In [ ]:
trump = pd.read_csv('/kaggle/input/us-election-2020-tweets/hashtag_donaldtrump.csv',na_values = missing_values, engine="python")

quick look at biden dataset

In [ ]:
biden.head()

quick look at trump dataset

In [ ]:
trump.head()

**Some columns are not so useful for our work so we'll remove them**

In [ ]:
biden = biden.drop(columns = ['created_at', 'tweet_id','source','user_id','user_name','user_screen_name','user_description','user_location','lat','long','city','state_code','collected_at']) 


In [ ]:
trump = trump.drop(columns = ['created_at', 'tweet_id','source','user_id','user_name','user_screen_name','user_description','user_location','lat','long','city','state_code','collected_at']) 


**Some rows contains null values. Since dataset is quite large we can manage to remove these rows**

In [ ]:
biden = biden.dropna(axis = 0, how = 'any') #it will remove all rows containing 'any' value as null(at least one)
print(len(biden)) #size after removing rows
print(biden.isnull().sum()) #count of null values in each column

**After removing unuseful columns and rows**

In [ ]:
biden.head()

**Same procedure for trump dataset**

In [ ]:
trump = trump.dropna(axis = 0, how = 'any')
print(len(trump))
print(trump.isnull().sum())

In [ ]:
#retwwet_count and user_followers_count columns conatins values as object/string
#so it'll be better to convert into float
biden = biden.astype({"retweet_count": float, "user_followers_count":float})
trump = trump.astype({"retweet_count": float, "user_followers_count":float})

**Below function will fill null values according to percentage of non-null values in that column**

In [ ]:
""" import random
def fill_missing(column):
    col = column.value_counts().to_dict()
    prob = random.choices(list(col.keys()), weights = list(col.values()), k=100)
    null_val = column.isnull()
    for i in range(len(null_val)):
        if null_val[i]== True:
            column[i] = prob[i%100]
    return column
"""

**Below function will make dictionary for certain column. Keys will be its elements and values will be count of elements in that columns**

In [ ]:
def make_dictionary(dataframe,column):
    dictt = {}
    for i in list(dataframe[column]):
        if i in dictt:
            dictt[i] = dictt.get(i) + 1
        else:
            dictt[i] = 1
    return dictt
    

Making dictionaries for certain columns

In [ ]:
country_biden = make_dictionary(biden,'country')
state_biden = make_dictionary(biden,'state')
continent_biden = make_dictionary(biden,'continent')
continent_biden

In [ ]:
country_trump = make_dictionary(trump,'country')
state_trump = make_dictionary(trump,'state')
continent_trump = make_dictionary(trump,'continent')
continent_trump

**Since there're so many keys in each dictionary, we can delete some keys with values less than certain threshold number**

In [ ]:
#all keys with values less than 1000 will be deleted
for i in list(country_biden):
    if country_biden.get(i)<1000:
        del country_biden[i]
#now arranging keys according to values in asc order
country_biden  = {k: v for k, v in sorted(country_biden.items(), key=lambda item: item[1])}
country_biden


In [ ]:
for i in list(country_trump):
    if country_trump.get(i)<1000:
        del country_trump[i]
country_trump  = {k: v for k, v in sorted(country_trump.items(), key=lambda item: item[1])}
country_trump


# EDA

In [ ]:
import matplotlib.pyplot as plt
p1 = plt.bar(np.arange(len(country_biden)),list(country_biden.values()),0.9)
plt.ylabel('tweets')
plt.title('tweets per country for biden')
plt.xticks(np.arange(len(country_biden)), country_biden.keys(),rotation='vertical')
plt.show()

In [ ]:
p11 = plt.bar(np.arange(len(country_trump)),list(country_trump.values()),0.9)
plt.ylabel('tweets')
plt.title('tweets per country for trump')
plt.xticks(np.arange(len(country_trump)), country_trump.keys(),rotation='vertical')
plt.show()

In [ ]:
state_biden

**finding correlation between winning in particular state with no. of tweets from that state**

In [ ]:
#1 for winning, 0 for losing in that state for biden
biden_won_states = {'Georgia':1, 'Arizona':1,'Florida':0,'Iowa':0,'Michigan':1,'Nevada':1,'New Hampshire':1,
                   'North Carolina':0,'Ohio':0,'Pennsylvania':1,'Texas':0,'Wisconsin':1,'Washington':1,
                   'Idaho':0,'Oregon':1,'Utah':0,'New Mexico':1,'Oklahoma':0,'Nebraska':0,
                   'Wyoming':0,'Montana':0,'North Dakota':0,'South Dakota':0,'Minnesota':1,'Illinois':1,
                   'Indiana':0,'West Virginia':0,'Virginia':1,'Maine':1,'Massachusetts':1,'Alabama':0,
                   'Mississippi':0,'Lousiana':0,'Michigan':1,'South Carolina':0,'Maryland':1,
                    'District Of Columbia':1,'Delaware':1,'New Jersey':1,'Connecticut':1,'Rhode Island':1,
                   'Vermont':1,'Arkansas':0,'Kansas':0,'Missouri':0,'Tennessee':0,'Kentucky':0}
st_x = [] #stores number of tweets from each state
st_y = [] #stores 1 for winning, 0 for losing in each state
for i in state_biden.keys():
    if i in biden_won_states.keys():
        st_x.append(state_biden.get(i))
        st_y.append(biden_won_states.get(i))
print(st_x)
print(st_y)
corr = np.corrcoef(st_x,st_y) #correlation b/w tweets and winning in particular state
print(corr)

**As you can see we got very less value for corr b/w tweets and winning. Now we'll try using vote percent instead of 0/1 and see the corr b/w tweets and vote percent**

In [ ]:
#vote percent of biden in each state
biden_won_states_percent = {'Georgia':49.5, 'Arizona':49.4,'Florida':47.9,'Iowa':45.0,'Michigan':50.6,'Nevada':50.1,'New Hampshire':52.8,
                   'North Carolina':48.7,'Ohio':45.2,'Pennsylvania':50.0,'Texas':46.5,'Wisconsin':49.6,'Washington':58.4,
                   'Idaho':33.1,'Oregon':56.9,'Utah':37.7,'New Mexico':54.3,'Oklahoma':32.3,'Nebraska':39.4,
                   'Wyoming':26.7,'Montana':40.5,'North Dakota':31.9,'South Dakota':35.6,'Minnesota':52.6,'Illinois':57.4,
                   'Indiana':41.0,'West Virginia':29.7,'Virginia':54.3,'Maine':52.9,'Massachusetts':65.6,'Alabama':36.6,
                   'Mississippi':40.5,'Lousiana':39.9,'Michigan':50.6,'South Carolina':43.4,'Maryland':65.4,
                    'District Of Columbia':92.9,'Delaware':58.8,'New Jersey':57.2,'Connecticut':59.3,'Rhode Island':59.6,
                   'Vermont':66.4,'Arkansas':34.8,'Kansas':41.3,'Missouri':41.3,'Tennessee':37.4,'Kentucky':36.2}
st_x_per = [] #stores tweets count of each state
st_y_per = [] #store vote percent of each state for biden
for i in state_biden.keys():
    if i in biden_won_states_percent.keys():
        st_x_per.append(state_biden.get(i))
        st_y_per.append(biden_won_states_percent.get(i))
print(st_x_per)
print(st_y_per)
corr1 = np.corrcoef(st_x_per,st_y_per)
print(corr1)

**we can see we got slightly more correlation using vote percent.**

In [ ]:
#deleting some keys with less values or state which are not in USA
for i in list(state_biden):
    if state_biden.get(i)<2000:
        del state_biden[i]
del state_biden['Istanbul']
del state_biden['Berlin']
del state_biden['Maharashtra']
del state_biden['Delhi']
del state_biden['Ontario']
del state_biden['Ile-de-France']
del state_biden['England']

state_biden  = {k: v for k, v in sorted(state_biden.items(), key=lambda item: item[1])} #sorting dict
state_biden

In [ ]:
for i in list(state_trump):
    if state_trump.get(i)<2500:
        del state_trump[i]
del state_trump['Istanbul']
del state_trump['Berlin']
del state_trump['Maharashtra']
del state_trump['Delhi']
del state_trump['Ontario']
del state_trump['Ile-de-France']
del state_trump['England']
del state_trump['North Rhine-Westphalia']
del state_trump['British Columbia']
del state_trump['Scotland']
state_trump  = {k: v for k, v in sorted(state_trump.items(), key=lambda item: item[1])}
state_trump

In [ ]:
p2 = plt.bar(np.arange(len(state_biden)),list(state_biden.values()),0.9)
plt.ylabel('tweets')
plt.title('tweets per state for biden')
plt.xticks(np.arange(len(state_biden)), state_biden.keys(),rotation='vertical')
plt.show()

In [ ]:
p3 = plt.bar(np.arange(len(state_trump)),list(state_trump.values()),0.9)
plt.ylabel('tweets')
plt.title('tweets per state for trump')
plt.xticks(np.arange(len(state_trump)), state_trump.keys(),rotation='vertical')
plt.show()

**deleting unuseful keys of continent dict**

In [ ]:
for i in list(continent_biden):
    if continent_biden.get(i)<100:
        del continent_biden[i]
continent_biden  = {k: v for k, v in sorted(continent_biden.items(), key=lambda item: item[1])}
continent_biden

In [ ]:
for i in list(continent_trump):
    if continent_trump.get(i)<100:
        del continent_trump[i]
continent_trump  = {k: v for k, v in sorted(continent_trump.items(), key=lambda item: item[1])}
continent_trump

**Plotting percentage of tweets from diff continents**

In [ ]:
#for biden
plt.pie([v for v in continent_biden.values()],labels = [k for k in continent_biden.keys()],autopct='%1.1f%%')
plt.show()

In [ ]:
#for trump
plt.pie([v for v in continent_trump.values()],labels = [k for k in continent_trump.keys()],autopct='%1.1f%%')
plt.show()

# Sentiment analysis

Combining all text from tweets into single string

In [ ]:
text_biden = " ".join(biden.tweet)
len(text_biden)

**dividing large text into lists each containing 10000 chars**

In [ ]:
n = 10000
text_list_biden = [text_biden[i:i+n] for i in range(0, len(text_biden), n)]


In [ ]:
len(text_list_biden) #total lists each containing 10000 chars

In [ ]:
text_trump = " ".join(trump.tweet)
n = 10000
text_list_trump = [text_trump[i:i+n] for i in range(0, len(text_trump), n)]
len(text_list_trump)

In [ ]:
# first, we import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# next, we initialize VADER so we can use it within our Python script
sid = SentimentIntensityAnalyzer()
scores_biden = {'compound': 0, 'neg': 0, 'neu': 0, 'pos': 0}
#compound for overall sentiment
for i in range(1000): #we're analysing only 1000 list for saving time instead of ~5000
    score = sid.polarity_scores(text_list_biden[0]) #sentiment score of each list
    #adding this score to total score
    scores_biden['compound'] = scores_biden['compound'] + score['compound']
    scores_biden['neg'] = scores_biden['neg'] + score['neg']
    scores_biden['neu'] = scores_biden['neu'] + score['neu']
    scores_biden['pos'] = scores_biden['pos'] + score['pos']
#since we've added scores of 1000 lists 
#divide each score by 1000 to get mean score
scores_biden['compound'] = scores_biden['compound']/1000
scores_biden['neg'] = scores_biden['neg'] /1000
scores_biden['neu'] = scores_biden['neu'] /1000
scores_biden['pos'] = scores_biden['pos'] /1000
scores_biden

**we got overall sentiment as -0.97 which indicates that mostly tweets are negative rather than positive**

In [ ]:
scores_trump = {'compound': 0, 'neg': 0, 'neu': 0, 'pos': 0}
for i in range(1000):
    score = sid.polarity_scores(text_list_trump[0])
    scores_trump['compound'] = scores_trump['compound'] + score['compound']
    scores_trump['neg'] = scores_trump['neg'] + score['neg']
    scores_trump['neu'] = scores_trump['neu'] + score['neu']
    scores_trump['pos'] = scores_trump['pos'] + score['pos']
scores_trump['compound'] = scores_trump['compound'] /1000
scores_trump['neg'] = scores_trump['neg'] /1000
scores_trump['neu'] = scores_trump['neu'] /1000
scores_trump['pos'] = scores_trump['pos'] /1000
scores_trump

**we got overall sentiment for trump as -0.99 which is more negative than biden(-0.97) which concludes that people are saying more negative things for trump.
There may be some tweets for #biden in which negative words are meant for trump and vice-versa. 
So we can get more accurate results by considering only those tweets which are only for either biden or trump.
But I think still there'll be more negative tweets for trump ;)**